# Tarea 5: Redes Recurrentes <br/> CC6204 Deep Learning, Universidad de Chile 

**Fecha de entrega: 30 de diciembre de 2020 ([Hoja de respuestas](https://colab.research.google.com/drive/1tCAAVMqGCIwy7c1P48IUuypqGVFYCUJs#scrollTo=HNoC8iTiKtg0))**

En esta tarea programarás arquitecturas de redes recurrentes y las probarás para dos casos: generación (incondicional) de texto, y generación de texto condicionado a imágenes de entrada. En ambos casos usaremos el conjunto de datos de [Filckr8k](http://hockenmaier.cs.illinois.edu/8k-pictures.html) introducido en [este artículo](https://www.aclweb.org/anthology/W10-0721/) que contiene imágenes y subtítulos (captions. Como en las tareas anteriores, no espearmos obtener resultado de estado del arte, sino más bien esperamos que se familiaricen con las arquitecturas necesarias para resolver problemas más complejos que mezclan texto e imágenes.

El material necesario para resolver esta tarea es el siguiente:
* [Video: Introducción a las Redes Recurrentes](https://www.youtube.com/watch?v=yHzflmQ9EoY)
* [Video: Arquitecturas de RNNs](https://www.youtube.com/watch?v=Bcy_no-u_BM)
* [Video: Autoregresión, Language Modelling y Seq-to-Seq](https://www.youtube.com/watch?v=bsKwb7wjYYc)
* [Video: RNNs con compuertas (GRU y LSTM)](https://www.youtube.com/watch?v=cDT9oYyXgjo)

También puede servirte repasar los temas de redes convolucionales de la [tarea anterior](https://colab.research.google.com/drive/1K18Vqh81UtJtKyZyO7LcPRgqLKl7rG8F).

(por Jorge Pérez, https://github.com/jorgeperezrojas, y Mauricio Romero, https://github.com/fluowhy)

# Preliminares: Redes Recurrentes en `pytorch`

Para esta tarea les permitimos usar todas las capacidades de `pytorch` para manejo de datos, cálculos de pérdidas, y sobre todo, redes recurrentes. En particuar recuerda los siguientes módulos:

* [`torch.optim`](https://pytorch.org/docs/stable/optim.html) para funcionalidades de optimizadores como SGD, Adam, RMSProp,
* [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) para funcionalidades para cargar datos, y
* [`torch.nn`](https://pytorch.org/docs/stable/nn.html) para funcionalidades para capas recurrentes, pooling, dropout, batch normalization, funciones de activación, pérdida, etc. (te recomendamos mirar con especial cuidado las clases [`torch.nn.RNN`](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html), [`torch.nn.GRU`](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html) y [`torch.nn.LSTM`](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html))


# Parte 1: Generación de texto

En esta parte crearás una red recurrente autoregresiva para predecir
la siguiente palabra dada una secuencia de palabras de entrada (para más detalles de autoregresión puedes ver [este video](https://youtu.be/Bcy_no-u_BM?t=3460) desde el minuto 58 para la idea general, y [este video](https://www.youtube.com/watch?v=bsKwb7wjYYc) hasta el minuto 32 para un detalle de cómo usarlo con oraciones y palabras).



### Datos

Lo primero importante es procesar los datos de entrenamiento. Para esto hemos dejado en el [repositorio del curso](https://github.com/dccuchile/CC6204/blob/master/2020/tareas/tarea5/utils.py) código para crear datasets de texto directamente desde los datos de Flickr8k. Hay varias funciones importantes que se usarán para para obtener (solo) los textos desde el dataset de Flickr8k, tokenizarlos (que esencialmente significa separar las palabras), construir el vocabulario (el conjunto de todos los tokens), asignar identificadores a cada uno de los tokens para trabajar con enteros en vez de strings, y usar todo esto para generar finalmente los dataset de entrenamiento, validación y prueba. En simple, el código sirve para pasar de una representación como esta

```
A woman holding a cup of tea.
```

a una como esta

```python
[0, 238, 94, 0, 1570, 9, 1022, 14]
```

En esta representación, hemos asignado el token `a` al identificador `0`, el token `woman` al identificador `238` y el token `.` al identificador `14`. Los identificadores dependerán siempre del vocabulario de cada aplicación, lo de arriba es solo para ejemplificar.

Puedes ver más detalles en la [hoja de respuestas](). Como siempre, **los códigos que les dejamos son para ayudarles, pero ustedes pueden generar sus propios códigos** para hacer estos procesamientos si así lo quieren (es un buen ejercicio!).

Un punto importante es que debemos considerar dos tokens especiales que se deben agregar al vocabulario
* `<sos>` para marcar el inicio de una secuencia de texto, y
* `<pad>` para marcar una posición que queremos que nuestro modelo ignore (más adelante explicamos su uso en detalle)

Los códigos que les dejamos disponibles, ya agregan estos dos tokens especiales (pero si tú haces tu propio código, debes considerarlos también).


### Datos *target* para entrenar

Una vez procesados los datos, nuestro dataset (entrenamiento, validación o prueba), debería verse simplemente como un conjunto de secuencias de identificadores. Nos queda solo un par de pasos que dedemos considerar antes de pensar en el diseño de nuestra red y en el entrenamiento.

Como nuestra red es autoregresiva, la misma secuencia de entrada se debe usar como datos de predicción. Supongamos que queremos entrenar nuestra red con la secuencia 
```
A woman holding a cup of tea .
``` 

En este caso supondremos que nuestra red recibe como entrada (los identificadores de) 
```
<sos> A woman holding a cup of tea
```
y que lo que espera ver en la salida son (los identificadores de)
```
A woman holding a cup of tea .
```

Hay dos cosas importantes de notar. Lo primero es que a la secuencia de entrada le estamos agregando el `<sos>` para indicar el inicio. Lo segundo es que nos estamos aprovechando del punto `.` para indicar el final de la oración. El código que les dejamos disponible se encarga de agregar `<sos>` y el punto (si es que no estuviera presente). Lo otro es notar que tanto la entrada como lo que se espera de salida son substrings específicos de la secuencia
```
<sos> A woman holding a cup of tea .
```
que como lista (tensor) de identificadores se vería similar a esto
```python
d = tensor([8460, 0, 238, 94, 0, 1570, 9, 1022, 14])
```
(nota que el identificador de `<sos>` en este caso sería `8460`.)

En nuestra nomenclatura estándar para redes neuronales, nuestro "$(x,y)$" (entrada y predicción) se puede construir entonces desde la secuencia de arriba de la siguiente forma:

```python
x = d[:-1]      # x = tensor([8460, 0, 238, 94, 0, 1570, 9, 1022]) 
y = d[1:]       # y = tensor([0, 238, 94, 0, 1570, 9, 1022, 14])
```

Debes tener muy claro esto cuando generes el código para entrenar (también si es que decides usar o modificar el código que dejamos disponible).

### Paquetes de ejemplos y *padding*

Finalmente, como en las tareas anteriores, querremos entrenar nuestras redes con paquetes de ejemplos. Esto nos pone una dificultad adicional dado que dos textos de entrada pueden tener largos diferentes. Acá es donde entra el concepto de padding. En simple, cuando generemos un paquete de ejemplos para poner todos los ejemplos en un mismo tensor, completaremos las secuencias más cortas usando el token especial `<pad>` que mencionamos más arriba. Imaginemos que queremos generar un paquete con los dos textos siguientes:
```
A woman holding a cup of tea .
A man with a dog .
``` 
En este caso, lo que realmente consideraremos serán los textos
```
<sos> A woman holding a cup of tea .
<sos> A man with a dog . <pad> <pad>
```
de esta forma ambas secuencias estarán compuestas por 9 tokens. Si suponemos que el token de padding tiene identificador `8459`, entonces el paquete de ejemplos que queremos considerar se vería como

```python
D = tensor(
  [[8460, 8460], 
   [   0,    0], 
   [ 238,   78], 
   [  94,   36], 
   [   0,    0], 
   [1570,   27], 
   [   9,   14], 
   [1022, 8459], 
   [  14, 8459]])
```
Nota que lo que estamos haciendo es un paquete que tiene dimensiones `(T,B)` donde `T` es el largo de la secuencia más larga del paquete (`9` en este caso) y `B` es el tamaño del paquete (`2` en este caso). Típicamente a este `T` se le llama *dimensión de tiempo* y el estándar en `pytorch` es considerarla como la primera dimensión. Esto puede ser un poco confuso al principio dado que en redes feed forward siempre consideramos al batch como la primera dimensión, pero para procesamiento más avanzado de secuencias con redes neuronales, es muy cómodo tener la dimensión de tiempo como la primera. Existen utilidades que simplifican la creación de paquetes de ejemplos y se preocupan de hacer el padding correcto, en particular la funcion [`pad_sequence`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html#torch.nn.utils.rnn.pad_sequence) puede generar el tensor anterior de manera muy simple
```python
d1 = tensor([8460, 0, 238, 94, 0, 1570, 9, 1022, 14]) 
d2 = tensor([8460, 0, 78, 36, 0, 27, 14])
D = pad_sequence([d1, d2], padding_value=8459)
``` 
lo que genera exactamente el tensor `D` que mostramos más arriba de dimensiones `(9,2)`. Desde `D` nuestros $X$ en $Y$ serían simplemente
```
X = D[:-1, :]
Y = D[1:, :] 
```
ambos de dimensiones `(T-1, B)`, en este caso `(8,2)`.

En la [hoja de respuesta](https://colab.research.google.com/drive/1tCAAVMqGCIwy7c1P48IUuypqGVFYCUJs#scrollTo=HNoC8iTiKtg0), usamos algo similar a `pad_sequence` cuando creamos el dataloader para nuestra red mediante la opción `collate_fn`. El uso debería estar claro en la hoja de respuestas, pero si tienes dudas del uso de `collate_fn` para un dataloader, te recomendamos mirar [la documentación](https://pytorch.org/docs/stable/data.html#automatic-batching-default). 

**Observación**: si no te acostumbras a tener la dimensión de tiempo como primera dimensión, puedes usar la del batch, pero debes tener cuidado de consideralo en todo tu código usando la opción `batch_first=True` en los llamados a funciones de pytorch que lo requieran. Por ejemplo, si arriba hubieramos hecho
```
D = pad_sequence([d1, d2], padding_value=8459, batch_first=True)
```
hubiéramos obtenido un tensor de dimensiones `(2,9)`.

### Largo de las secuencias

Para algunas aplicaciones puede ser cómodo saber el largo de cada una de las secuencias de entrenamiento, validación y prueba. Por eso, tanto los `dataset` como `dataloader` que dejamos en la [hoja de respuesta](https://colab.research.google.com/drive/1tCAAVMqGCIwy7c1P48IUuypqGVFYCUJs#scrollTo=HNoC8iTiKtg0), no solo entregan las secuencias mismas, sino también los largos. En específico, la forma de usarlos es como sigue:
```python
# train_dataset y train_daloader son creados con las funciones utilitarias
d, length = train_dataset[0]
D, Lengths = next(iter(train_dataloader))
```
En este caso, `d` sería una única secuencia (la primera del dataset de entrenamiento) y `length` es el largo de esa secuencia. Por su parte `D` sería un paquete de secuencias con padding (de dimensiones `(T,B)`) y `Lengths` sería un tensor de dimensión `(B)` con los largos de cada una de las secuencias del paquete.  


## 1a) Red recurrente

En esta parte implementarás una red recurrente usando las clases que pytorch provee para esto. En particular, debes usar al menos tres componentes: 
1. una **capa de embeddings** que convierta los índices de cada token en un vector de características,
2. una **capa recurrente** que procese la secuencia de vectores que resultan desde la capa de embeddings, y
3. una o varias capas para **generar la salida** de la red en cada instante de tiempo.

Conceptualmente, los tres pasos anteriores los [explicamos en clases](https://www.youtube.com/watch?v=bsKwb7wjYYc). Para poder programarlos en pytorch debes usar las siguientes clases.

1. Para la capa de embeddings debes usar [`torch.nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) indicando el tamaño del vocabulario (`vocab_size` en el código de abajo), las dimensiones del embedding (`emb_dim`) y opcionalmente, el índice del token de padding (`pad_idx`). 
```python
emb_layer = Embedding(voc_size, emd_dim, padding_idx=pad_idx)
```
Un punto importante es que la capa de embedding se puede aplicar sin problemas a un tensor de cualquier rango que contenga índices de tokens y la salida será un tensor con una dimensión adicional. Por ejemplo, si consideramos el tensor `X` de mas arriba (de dimensiones `(8,2)`) y ejecutamos el siguiente código:
```python
emb_layer = Embedding(10000, 50, padding_idx=8459)
E = emb_layer(X)
```
el tensor `E` tendrá dimensiones `(8,2,50)`. El índice del token de padding se usa esencialmente para ahorrar computación (dejando todos los embeddings correspondientes como vectores de ceros), pero no tiene efecto en las dimensiones de salida.

2. Para la capa recurrente, puedes usar las clases [`torch.nn.RNN`](https://pytorch.org/docs/stable/generated/torch.nn.RNN) o [`torch.nn.GRU`](https://pytorch.org/docs/stable/generated/torch.nn.GRU) o [`torch.nn.LSTM`](https://pytorch.org/docs/stable/generated/torch.nn.LSTM) indicando la dimension del vector de entrada (el que en clases llamamos $x^{\langle t\rangle}$), la dimensión del estado recurrente (el que en clases llamamos $h^{\langle t\rangle}$), y opcionalmente la cantidad de capas recurrentes para apilar. Por ejemplo, el siguiente código genera tres capas recurrentes simples con `50` como dimensión de entrada y `30` como dimensión del estado recurrente:
```python
rnn_layer = RNN(50, 30, num_layers=3)
```
Para el caso de las capas recurrentes en pytorch, la salida siempre es un par en donde la primera componente del par es la secuencia completa de los estados recurrentes de la capa final para cada instante de tiempo, o sea, lo que en clases llamamos $h^{\langle 1\rangle(k)}, h^{\langle 2\rangle(k)}, \ldots, h^{\langle N\rangle(k)}$ donde estoy apilando $k$ capas recurrentes. La segunda componente depende de la red recurrente específica que estes usando, pero en general representa al estado en el último instante de tiempo de la red para cada una de las capas apiladas (lo que en clases llamamos $h^{\langle N\rangle(1)}, h^{\langle N\rangle(2)}, \ldots, h^{\langle N\rangle(k)}$ para $k$ capas apiladas). Por ejemplo, si aplicamos la capa `rnn_layer` al tensor `E` de más arriba como en el siguiente código
```python
H, S = rnn_layer(E)
```
el tensor `H` tendrá dimensiones `(8,2,30)` (es decir, tiempo x paquete x dimensión de salida), mientras que el tensor `S` tendrá dimensiones `(3,2,30)` (en donde el `3` aparece por la cantidad de capas que tenemos apiladas en la `rnn_layer`, es decir sera de capas x paquete x dimensión de salida). El caso de las `LSTM` es un poco distinto porque debe considerar además la celda de memoria en `S` (puedes ver [la documentación acá](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)). Un punto (muy) importante es que las capas recurrentes en pytorch, además de recibir el tensor al que le deben calcular la salida, necesitan el estado inicial de la recurrencia (lo que en clases llamamos $h^{\langle 0\rangle}$). Si este estado no se entrega (`None`), entonces se asume como un vector de solo ceros. Cuando construyas tu red completa, en la función `forward` procura aceptar un atributo opcional para entregar este estado inicial a la red recurrente.

3. Para la capa de salida puedes usar capas lineales (una o más) con activaciones si es que tienes capas intermedias. Lo único imprescindible es que la capa final debe tener como dimensión el tamaño del vocabulario de entrada (recuerda que estaremos prediciendo palabras de un vocabulario). Por ejemplo este código usa solo una capa lineal y se la aplica directamente al tensor `H`
```python
out_layer = Linear(30, 10000) 
logits = out_layer(H)
```
En este caso el tensor de salida `logits` tendrá dimensiones `(8,2,10000)` y contendrá los logits (los valores antes de aplicar softmax) y que son los que se deben usar para computar la pérdida (siguiente sección).

Con esto ya deberías ser capaz de diseñar y programar tu primera red recurrente que se usará como red autoregresiva para procesar texto. Para esto completa el código de abajo.

In [ ]:
# Acá empieza tu código

class RedRecurrente(torch.nn.Module):
    def __init__(self, ...): # Piensa en todo lo que necesitas para incializar.
        # Crea las capas considerando al menos los puntos de arriba.
        pass    

    def forward(self, x, h_0=None):
        # Acá debes programar la pasada hacia adelante.
        # El vector h_0 deberías simplemente pasarlo directo
        # a tu red recurrente (RNN, o GRU, o LSTM) y será necesario
        # para trabajar en la sección (1c) y en la parte 2. 
        # También puedes usar dropout, batch normalization o lo que necesites.
        ...
        return logits

    # Todas las otras funciones que necesites

## 1b) Entrenamiento

En esta parte debes programar el loop de entrenamiento de tu red. Será muy parecido a como lo has hecho para tareas anteriores con la salvedad de que debes generar los batches `X` e `Y` como lo explicamos más arriba (desde lo que te entrega el dataloader que dejamos disponible).

Un punto muy importante es cómo computar la pérdida. Dado que estaremos haciendo clasificación, la pérdida que usaremos será [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Debes tener cuidado con un par de cosas. Lo primero es que en el paquete de entrada muy posiblemente hayan posiciones que deben ser ignoradas cuando computes la pérdida. Para esto la clase `CrossEntropyLoss` de pytorch nos da la opción de definir un índice a ignorar (`ignore_index`). La idea es que cada vez que se encuentre con ese índice en los datos de predicción (`target`) esa posición no se usará para computar la pérdida y por lo tanto no incidirá en el *backpropagation*. Lo segundo es que, dado que calcularemos la entropía cruzada para varias posiciones, tenemos que especificar cómo se combina el resultado. Para esto `CrossEntropyLoss` nos da la opción `reduction` que puede ser `mean` o `sum` (también puede ser `none` en cuyo caso no combinará los resultados).

Finalmente, y posiblemente el punto más importante: la clase `CrossEntropyLoss` espera como entrada dos tensores `logits` y `target` con dimensiones específicas. El primero debe ser un tensor de dimensiones `(N,C)` donde `C` es la cantidad de clases y `N` son todas las *posiciones* para las que debemos calcular el *softmax* y posteriormente la entropía cruzada. En nuestro caso, la cantidad de clases es el tamaño del vocabulario. El segundo tensor, `target`, debe ser de dimensión `(N)` y contiene los índices que esperamos predecir. Dado que en nuestro caso tenemos varias posiciones para cada ejemplo del paquete, debemos redimensionar los tensores antes de entregarlos a `CrossEntropyLoss`.

El siguiente código muestra un ejemplo completo de qué deberíamos hacer para computar correctamente la pérdida. Incluimos los mismos códigos anteriores para que veas cómo se haría siguiendo todos los pasos
```python
from torch import tensor
from torch.nn import Embedding, RNN, Linear, CrossEntropyLoss
from torch.nn.utils.rnn import pad_sequence

# indice de padding y tamaño de vocabulario de ejemplo
pad_idx = 8459
voc_size = 10000

# datos de ejemplo
d1 = tensor([8460, 0, 238, 94, 0, 1570, 9, 1022, 14]) 
d2 = tensor([8460, 0, 78, 36, 0, 27, 14])
D = pad_sequence([d1, d2], padding_value=pad_idx)
X, Y = D[:-1, :], D[1:, :]

# capas
emb_layer = Embedding(voc_size, 50, padding_idx=pad_idx)
rnn_layer = RNN(50, 30, num_layers=3)
out_layer = Linear(30, voc_size) 

# aplica las capas a los datos
E = emb_layer(X)
H, S = rnn_layer(E)
logits = out_layer(H)

# función de error considerando padding y promedio (mean)
loss_fn = CrossEntropyLoss(ignore_index=pad_idx, reduction='mean')

# redimensiona los logits para dejarlos de dimensiones (N,voc_size)
logits = logits.view(-1, voc_size)

# redimensiona Y para dejarlo de dimensión (N)
target = Y.view(-1)

# ahora podemos aplicar la función de loss sin problemas
L = loss_fn(logits, target)
```

Ya sabes todo lo necesario para crear el loop de entrenamiento y prueba. Crea código para entrenar tu red por una cantidad de épocas que lleven a un valor de la pérdida que consideres razonable. Dadas las características de la tarea, la evaluación (en el conjunto de validación) la debes hacer usando la misma función de entropía cruzada (es decir, no tendremos métricas específicas como el "acierto" que ocupábamos en tareas anteriores). Reporta el valor de la pérdida según las épocas. Ideal es si puedes hacer comparaciones con distintos tipos de redes.

**Observación**: En el código de utilidades que les dejamos en el sitio del curso, hay partes que pueden reutilizar para el loop de entrenamiento (`train_one_epoch` y `eval_one_epoch`), pero por favor consideren que los loops de entrenamiento dependen mucho de cómo están creando y usando las redes, por lo que posiblemente deban adaptarlos, o mejor aun, crear sus propios loops inspirándose en los que están en el archivo. En cualquier caso, ustedes son l@s responsables de que sus loops de entrenamiento funcionen.

In [ ]:
# Acá tu código para el loop de entrenamiento
# y los gráficos de la pérdida

## 1c) Generación de texto

Usa tu red entrenada para generar texto de manera autoregresiva. Para esto debes crear una función `generate_sentence` que reciba (al menos) el modelo y una oración parcial que se irá completando de manera autoregresiva. Usa lo que has aprendido en clases y lo que practicaste programando y entrenando tu red para decidir la mejor forma de generar texto. Toma en cuenta que si siempre eliges el token con la mayor probabilidad entonces la salida de tu red será poco diversa. Por esto sería mejor que uses un "sampleo" que en este caso podrías hacerlo de manera muy simple usando la distribución [`Categorical`](https://pytorch.org/docs/stable/distributions.html#torch.distributions.categorical.Categorical) de pytorch. Esta distribución tiene la opción de entregarle los `logits` lo que simplifica la elección con un código como este:
```python
logits = ... # calcula los logits para el vocabulario usando el modelo
token_id = Categorical(logits=logits).sample() # esto elige un token desde los logits
```
Tu función debería generar hasta cierta condición de término. Lo más natural es que generes hasta que el token de salida sea `.` (punto), o hasta que llegue a un largo máximo. Muestra un par ejemplos de oraciones generadas por tu modelo.

In [ ]:
# Acá tu código para generar texto usando el modelo

def generate_sentence(model, init_sentence, ...):
  # Usa acá lo que necesites para crear una secuencia de
  # salida. Muy posiblemente tendrás que usar un tokenizador
  # y el diccionario para pasar de índices a tokens (palabras).
  return sentence

## 1d) Opcional: *Beam Search*

Otra forma de generar texto de buena calidad es usando *Beam Search* que es una técnica que relaja un poco la idea de siempre obtener el token de probabilidad máxima en cada paso de generación. En simple, lo que hace beam search es considerar un valor de *ancho del beam*, digamos `K`. En cada instante de tiempo de la autoregresión tenemos `K` secuencias candidatas de salida, cada una de ellas con un *score* correspondiente a la multiplicación de las probabilidades que entrega el modelo para cada token de cada secuencia. En cada paso, desde cada una de las `K` secuencias, se eligen los `K` tokens siguientes más probables lo que deja un total de `K*K` secuencias. De ellas se eligen las `K` secuencias de mayor *score* y se continúa con el proceso.

En la práctica multiplicar las probabilidades de los tokens de cada secuencia puede dar un número demasiado pequeño, por lo que es natural, en vez de multiplicar las probabilidades, sumar los logartimos de las probabilidades y usar eso como *score*. Valores pequeños de `K` (hasta 10, por ejemplo) ya dan buenos resultados.

En esta parte debes programar el método de *beam search* según la descripción mencionada. Muestra también un par ejemplos de oraciones generadas por tu modelo.

In [ ]:
# Acá tu código para generar texto usando beam search

def beam_search_generation(model, init_sentence, K, ...):
  # El K representa al ancho del beam para la búsqueda.
  return sentence

# Parte 2 (Opcional): Subtitulado de imágenes

En esta parte combinarás redes convolucionales y recurrentes para producir una arquitectura completa que pueda tomar una imagen y producir una descripción para la imagen. Deberías usar todo lo que has aprendido hasta ahora tanto de redes convolucionales como de recurrentes. En términos simples, tu arquitectura debería procesar una imagen y convertirla en un vector de características que luego debes usar para generar la secuencia de tokens de salida. Una forma directa de hacer esta parte sería con los siguientes pasos.

1. Primero puedes considerar un modelo pre-entrenado de red convolucional (como alguno de los que [aparecen acá](https://pytorch.org/docs/stable/torchvision/models.html)). Por ejemplo, para obtener la versión pre-entrenada de VGG16 puedes hacer
```python
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)
```

2. Usando el modelo puedes obtener características  a partir de las imágenes de entrada. Por ejemplo, si tienes un paquete de imágenes de entrada, digamos `Img`, puedes obtener sus características haciendo
```python
F = vgg16.features(Img)
```
Es muy importante considerar que la dimensión del tensor resultante dependerá de la red en particular que estés usando y de las dimensiones del tensor de entrada. En el caso de `vgg16`, si las imágenes de entrada son de `224x224`, es decir, si `Img` es de dimensiones `(B,3,224,224)`, entonces el tensor `F` será de dimensiones `(B,512,7,7)`. En cambio, si las imágenes de entrada son de `112x112` entonces el tensor de salida tendrá dimensiones `(B,512,3,3)`.  

3. A partir de la salida del modelo pre-entrenado, puedes usar una capa lineal para convertir las características anteriores en un vector que pueda ser entregado como estado inicial ($h^{\langle 0\rangle}$) a la parte recurrente.

4. Finalmente puedes usar tu código de red recurrente de la parte anterior para, dado el estado inicial computado en el paso (3), generar una secuencia de salida.

Algo importante es qué partes de las de arriba entrenas y cuáles no. Si confiamos en que un modelo de red convolucional pre-entrenado servirá para esta tarea (es una buena idea confiar en eso), entonces lo natural sería entrenar solo la parte de la red recurrente. Para esto debes asegurarte de que el modelo de extracción de características de la imagen esté en modo de evaluación y que no se computen gradientes mientras extrae las características. Esto lo puedes lograr haciendo algo como esto:
```python
vgg16.eval()
with torch.no_grad():
    F = vgg16.features(Img)
# continuar con la red recurrente fuera del `with torch.no_grad()`  
``` 
También recuerda pasar el modelo a la GPU para hacer los cómputos.

Los puntos de arriba son una primera aproximación, pero en esta parte te damos toda la libertad para que generes tu red. En particular podrías probar distintas cosas como entrenar una red convolucional específicamente para esta tarea. Otra cosa que puede resultar muy bien es pasarle las características de la imagen a la red recurrente no solo como estado inicial si no que combinarlo con cada paso recurrente (por ejemplo al momento de producir la salida). 

Ya tienes todas las herramientas para hacer la red como quieras. Crea la red, el loop de entrenamiento, y grafica la pérdida con respecto a las épocas. Finalmente usa tu red para generar texto desde las imágenes del conjunto de prueba y muestra un par de ejemplos.

## 2a) Red convolucional + recurrente


In [ ]:
class CaptioningModel(torch.nn.Module):
    def __init__(self, ...): 
        # Crea las capas considerando una parte que procese debe procesar
        # la imagen de entrada y otra que debe producir el texto (índices)
        # de salida.
        pass
        
    def forward(self, ...):
        # Acá debes programar la pasada hacia adelante.
        # Debes decidir qué le pasarás a la red y cómo haras la 
        # computación hacia adelante. Considera que no solo
        # debes entrenar los parámetros sino que además debes
        # después ser capaz de generar una secuencia de salida
        # desde una imagen de entrada.
        return ...        

## 2b) Entrenamiento 

In [ ]:
# Acá tu código para el loop de entrenamiento
# y los gráficos de la pérdida

## 2c) Generando texto desde imágenes de prueba

In [ ]:
# Acá tu código para generar texto usando desde imágenes
# y un par de ejemplos con las imágenes del conjunto de prueba